# python-database-CR_U_D-read-command-process-walk-through-python-mysql-1
> [TABLE OF CONTENTS](https://github.com/E6985/E2746/blob/master/README.md)
***

```sql
mysql> use school;
Database changed
```

```sql
mysql> show tables;
+------------------+
| Tables_in_school |
+------------------+
| subject          |
| teacher          |
+------------------+
2 rows in set (0.00 sec)
```

```sql
mysql> SELECT *
    -> FROM teacher;
+-----+--------------+-------+------------+------------+
| tid | Name         | level | experience | dob        |
+-----+--------------+-------+------------+------------+
|   1 | Mr. Pasteur  | L     |         15 | 1960-02-02 |
|   2 | Ms. Dubois   | L     |         22 | 1967-09-02 |
|   3 | Ms. Smith    | J     |          4 | 1980-03-23 |
|   4 | Mr. Hawking  | L     |         40 | 1951-02-19 |
|   5 | Mr. Kavanagh | J     |         50 | 1949-11-01 |
|   6 | Mr. Picasso  | J     |         42 | 1939-03-30 |
|   7 | Fr. Lynch    | L     |         55 | 1939-03-31 |
+-----+--------------+-------+------------+------------+
7 rows in set (0.00 sec)
```

```sql
mysql> SELECT  *
    -> FROM subject;
+-----------+--------------+---------------+
| Name      | Teacher      | OnLeavingCert |
+-----------+--------------+---------------+
| Biology   | Mr. Pasteur  |             1 |
| Colouring | Mr. Picasso  |             0 |
| English   | Mr. Kavanagh |             1 |
| French    | Ms. Dubois   |             1 |
| Maths     | Mr. Hawking  |             1 |
| Religion  | Fr. Lynch    |             1 |
| Spelling  | Ms. Smith    |             0 |
+-----------+--------------+---------------+
7 rows in set (0.00 sec)
```

- ask user to enter details of a new subject and adds it to the database [7]

```
$ vi py/schoolUpdateDB.py
```

```python
import pymysql

# define connection - name - conn
conn=None

# define function - fConnectDB
def fConnectDB():
    # accessible any function
    global conn
    conn=pymysql.connect(
    	host="localhost",
    	user="root",
    	password="******",
    	db="school",
    	cursorclass=pymysql.cursors.DictCursor)

# define function - fAddSubject
def  fAddSubject(nParN,nParT,nParLC):
    # check database connection
    if (not conn):
        # function fConnectDB - call
        fConnectDB() # connect database - school

	# parameter - %s - %s - %s -  nParN - nParT - nParLC
    nInsert="INSERT INTO subject (Name,Teacher,OnLeavingCert) VALUES(%s,%s,%s)"

    # close when finished
    with conn:
        # create object - cursor on connection
        cursor=conn.cursor()
        # execute sql - nQuery
        nCheckUpdate=cursor.execute(nInsert,(nParN,nParT,nParLC))
        print(nCheckUpdate)
```

```
$ vi py/python-mysql-1.py
```

```python
import schoolUpdateDB

# define function - main
def main():
	nName=input("Enter Subject:  ")
	nTeacher=input("Enter Teacher: ")
	nLOC=input("Enter on Leaving Cert <1/0>: ")

	# function schoolUpdateDB.fAddSubject - call
	schoolUpdateDB.fAddSubject(nName,nTeacher,nLOC)

# keyword - __name__ - evaluate current module [1]
if __name__=="__main__":
    # function main - call
    main()
```

```
$ python py/python-mysql-1.py
```

```python
Enter Subject:  New Subject
Enter Teacher: New Teacher
Enter on Leaving Cert <1/0>: 1
1
```

```sql
mysql> SELECT *
    -> FROM subject;
+-------------+--------------+---------------+
| Name        | Teacher      | OnLeavingCert |
+-------------+--------------+---------------+
| Biology     | Mr. Pasteur  |             1 |
| Colouring   | Mr. Picasso  |             0 |
| English     | Mr. Kavanagh |             1 |
| French      | Ms. Dubois   |             1 |
| Maths       | Mr. Hawking  |             1 |
| New Subject | New Teacher  |             1 |
| Religion    | Fr. Lynch    |             1 |
| Spelling    | Ms. Smith    |             0 |
+-------------+--------------+---------------+
8 rows in set (0.00 sec)
```

```
$ python py/python-mysql-1.py
```

```python
Enter Subject:  Another Subject
Enter Teacher: Another Teacher
Enter on Leaving Cert <1/0>: 0
1
```

```sql
mysql> SELECT *
    -> FROM subject;
+-----------------+-----------------+---------------+
| Name            | Teacher         | OnLeavingCert |
+-----------------+-----------------+---------------+
| Another Subject | Another Teacher |             0 |
| Biology         | Mr. Pasteur     |             1 |
| Colouring       | Mr. Picasso     |             0 |
| English         | Mr. Kavanagh    |             1 |
| French          | Ms. Dubois      |             1 |
| Maths           | Mr. Hawking     |             1 |
| New Subject     | New Teacher     |             1 |
| Religion        | Fr. Lynch       |             1 |
| Spelling        | Ms. Smith       |             0 |
+-----------------+-----------------+---------------+
9 rows in set (0.00 sec)
```

```
$ python py/python-mysql-1.py
```

```python
Enter Subject:  Another Subject
Enter Teacher: Another Teacher
Enter on Leaving Cert <1/0>: 0
...
...
...
pymysql.err.IntegrityError: (1062, "Duplicate entry 'Another Subject' for key 'subject.PRIMARY'")
```

```
$ vi py/python-mysql-1.py
```

```python
import schoolUpdateDB

# define function - main
def main():
	nName=input("Enter Subject:  ")
	nTeacher=input("Enter Teacher: ")
	nLOC=input("Enter on Leaving Cert <1/0>: ")

	try:
		# function schoolUpdateDB.fAddSubject - call
		schoolUpdateDB.fAddSubject(nName,nTeacher,nLOC)
	except:
		print("Error -",nName,"Already Exists")

# keyword - __name__ - evaluate current module [1]
if __name__=="__main__":
    # function main - call
    main()
```

```
$ python py/python-mysql-1.py
```

```python
Enter Subject:  Maths
Enter Teacher: Mr. Smith
Enter on Leaving Cert <1/0>: 1
Error - Maths Already Exists
```

- no distinction betweeen errors - only printing out [7]

```python
	try:
		# function schoolUpdateDB.fAddSubject - call
		schoolUpdateDB.fAddSubject(nName,nTeacher,nLOC)
	except:
		print("Error -",nName,"Already Exists")
```

[pymysql - exceptions](https://github.com/PyMySQL/PyMySQL/blob/master/pymysql/err.py)

```
$ vi py/python-mysql-1.py
```

```python
import schoolUpdateDB

# define function - main
def main():
	nName=input("Enter Subject:  ")
	nTeacher=input("Enter Teacher: ")
	nLOC=input("Enter on Leaving Cert <1/0>: ")

	try:
		# function schoolUpdateDB.fAddSubject - call
		schoolUpdateDB.fAddSubject(nName,nTeacher,nLOC)
	except Exception as nE:
		print("Error -",nE)

# keyword - __name__ - evaluate current module [1]
if __name__=="__main__":
    # function main - call
    main()
```

```
$ python py/python-mysql-1.py
```

```python
Enter Subject:  Maths
Enter Teacher: Mr. Smith
Enter on Leaving Cert <1/0>: 1
Error - (1062, "Duplicate entry 'Maths' for key 'subject.PRIMARY'")
```

```
$ vi py/python-mysql-1.py
```

```python
import  pymysql
import schoolUpdateDB

# define function - main
def main():
	nName=input("Enter Subject:  ")
	nTeacher=input("Enter Teacher: ")
	nLOC=input("Enter on Leaving Cert <1/0>: ")

	try:
		# function schoolUpdateDB.fAddSubject - call
		schoolUpdateDB.fAddSubject(nName,nTeacher,nLOC)
	except pymysql.err.IntegrityError as nE:
		print("Subject Already Exists -",nE)
	# catch all errors
	except Exception as nE:
		print("Generic ERROR",nE)

# keyword - __name__ - evaluate current module [1]
if __name__=="__main__":
    # function main - call
    main()
```

```
$ python py/python-mysql-1.py
```

```python
Enter Subject:  Maths
Enter Teacher: Mr. Smith
Enter on Leaving Cert <1/0>: 1
Subject Already Exists - (1062, "Duplicate entry 'Maths' for key 'subject.PRIMARY'")
```

***
# END